# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file_df = pd.read_csv("../output_data/weatheroutput.csv")
vacation_df = file_df.drop(columns=["Unnamed: 0"])
vacation_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Biak,98,ID,1584394243,71,-0.91,122.88,80.87,5.70
1,Cabo San Lucas,20,MX,1584393951,47,22.89,-109.91,82.99,6.93
2,Kapaa,90,US,1584394246,94,22.08,-159.32,73.40,26.40
3,Ikot Ekpene,100,NG,1584394247,77,5.18,7.71,82.49,7.99
4,Airai,46,TL,1584393660,83,-8.93,125.41,64.13,3.80


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Latitude' and 'Longitude' into  locations. 
locations = vacation_df[["Lat", "Lng"]].astype(float)

# Convert Humidity to float and store
# Also, handle NaN values
vacation_df = vacation_df.dropna()
humidity = vacation_df["Humidity"].astype(float)

In [5]:
lat = vacation_df["Lat"].fillna(0)
lng = vacation_df["Lng"].fillna(0)
coordinates = list(zip(lat, lng))

In [10]:
# Customize the size of the figure
figure_layout = {
    'width': '900px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Create heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity,
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

In [11]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(coordinates)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

In [12]:
# Plot Heatmap
# Assign the marker layer to a variable
heat_layer = gmaps.heatmap_layer(coordinates, weights=vacation_df["Humidity"])
# Add the layer to the map
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

Narrow down the DataFrame to find your ideal weather condition. For example:

A max temperature lower than 80 degrees but higher than 70.

Wind speed less than 10 mph.

Zero cloudiness.

Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

In [13]:
# Create new dataframe
ideal_weather_df = pd.DataFrame(vacation_df, columns = ["City", "Country", "Lat", "Lng", "Max Temp", "Wind Speed", "Cloudiness"])

# filter down to ideal weather conditions
max_temp = (ideal_weather_df["Max Temp"] <= 85) & (ideal_weather_df["Max Temp"] > 75)
wind_speed = (ideal_weather_df["Wind Speed"] < 10)
cloudiness = (ideal_weather_df["Cloudiness"] == 0)

ideal_weather_df[max_temp & wind_speed & cloudiness]

,City,Country,Lat,Lng,Max Temp,Wind Speed,Cloudiness
36,Berbera,SO,10.44,45.01,79.07,8.50,0
51,Bermejo,AR,-32.88,-68.80,84.20,8.05,0
53,Saldanha,ZA,-33.01,17.94,75.20,8.05,0
70,Guhāgar,IN,17.48,73.19,81.05,9.73,0
92,Morón,AR,-34.65,-58.62,79.00,5.82,0
216,Waingapu,ID,-9.66,120.26,83.93,5.55,0
300,Santa Vitória,BR,-18.84,-50.12,79.79,3.67,0
446,Necochea,AR,-38.55,-58.74,75.99,3.00,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
# Store into variable
hotel_df = pd.DataFrame(ideal_weather_df[max_temp & wind_speed & cloudiness])

hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Max Temp,Wind Speed,Cloudiness,Hotel Name
36,Berbera,SO,10.44,45.01,79.07,8.50,0,
51,Bermejo,AR,-32.88,-68.80,84.20,8.05,0,
53,Saldanha,ZA,-33.01,17.94,75.20,8.05,0,
70,Guhāgar,IN,17.48,73.19,81.05,9.73,0,
92,Morón,AR,-34.65,-58.62,79.00,5.82,0,
216,Waingapu,ID,-9.66,120.26,83.93,5.55,0,
300,Santa Vitória,BR,-18.84,-50.12,79.79,3.67,0,
446,Necochea,AR,-38.55,-58.74,75.99,3.00,0,


In [15]:
# Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_request = requests.get(base_url, params=params)

    # convert to json
    hotel_response = hotel_request.json()
    
    hotel_df.loc[index, "Hotel Name"] = hotel_response["results"][0]["name"]


In [16]:
hotel_df

,City,Country,Lat,Lng,Max Temp,Wind Speed,Cloudiness,Hotel Name
36,Berbera,SO,10.44,45.01,79.07,8.50,0,Damal Hotel
51,Bermejo,AR,-32.88,-68.80,84.20,8.05,0,Park Hyatt Mendoza
53,Saldanha,ZA,-33.01,17.94,75.20,8.05,0,Hoedjiesbaai Hotel
70,Guhāgar,IN,17.48,73.19,81.05,9.73,0,Nilambari Beach Resort
92,Morón,AR,-34.65,-58.62,79.00,5.82,0,Hotel Cabildo
216,Waingapu,ID,-9.66,120.26,83.93,5.55,0,Padadita Beach Hotel
300,Santa Vitória,BR,-18.84,-50.12,79.79,3.67,0,HOTEL GUIMA
446,Necochea,AR,-38.55,-58.74,75.99,3.00,0,Hotel Real


Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.

In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
output_hotel = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…